# Benchmark for Community detection

In [1]:
"""Load/import helper functions"""

import time
import random
from LocalPopular import locally_popular_clustering_with_hop_distance, extract_labels_from_communities, time_tester, calculate_scores_CD

from GraphFunctions import generate_agents, calculate_euclidian_relationships, create_graph, \
    my_make_circles, create_graphs_euclid, create_graphs_kNN, \
    generate_graph,create_graphs_hop_distance, create_graphs_hop_distance_abs,randomize_graph_node_labels

from PlotHelperFunctions import plot_clustering, plot_stuff

from sklearn.cluster import KMeans, DBSCAN
from sklearn.datasets import make_moons
from sklearn.metrics import rand_score
import numpy as np
import networkx as nx
from scipy.spatial import distance

from community_detection.leiden import leiden
from community_detection.louvain import louvain
from community_detection.quality_functions import CPM, Modularity


import data.jazz as jazz
import data.cora as cora

## Create Graphs

In [2]:
repetitions = 1    #Number of random isomorph permutation for each graph

cora_graph = cora.get_graph()

cora_graph = nx.relabel_nodes(cora_graph, {list(cora_graph.nodes())[i] : i for i in range(len(cora_graph.nodes()))} )
cora_truth = list(cora_graph.nodes[i]['subject'] for i in range(len(cora_graph.nodes())))
cora_perm_graph = []
cora_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(cora_graph,cora_truth)
    cora_perm_graph += [g]
    cora_perm_truth += [t]


jazz_graph = jazz.get_graph()
jazz_graph = nx.relabel_nodes(jazz_graph, {i : i-1 for i in range(len(jazz_graph)+1)} )
jazz_graph,_ = randomize_graph_node_labels(jazz_graph,None)
jazz_truth = None

jazz_perm_graph = []
jazz_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(jazz_graph,jazz_truth)
    jazz_perm_graph += [g]
    jazz_perm_truth += [t]


karate_graph = nx.karate_club_graph()
#karate_graph,_ = randomize_graph_node_labels(karate_graph,None)
karate_truth = list(karate_graph.nodes[i]["club"] for i in range(34))

karate_perm_graph = []
karate_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(karate_graph,karate_truth)
    karate_perm_graph += [g]
    karate_perm_truth += [t]

karate_perm_graph = [karate_graph,karate_graph,karate_graph,karate_graph,karate_graph]
karate_perm_truth = [karate_truth,karate_truth,karate_truth,karate_truth,karate_truth]

graph,graph_truth = generate_graph(10,25,0.2,0.05)

graph_perm_graph = []
graph_perm_truth = []
for i in range(repetitions):
    g,t = randomize_graph_node_labels(graph,graph_truth)
    graph_perm_graph += [g]
    graph_perm_truth += [t]




## Run the algorithms


In [3]:
import itertools
import timeit

f = 0.2   #f-bound
e = 0.2   #e-bound


graphs = [cora_perm_graph,karate_perm_graph,jazz_perm_graph,graph_perm_graph]
expected_clusters = [7,2,None,25]
graph_names = ['Cora', 'Karate Club','Jazz','25 quasi cliques (Name WIP)']
graph_truths =  [cora_perm_truth,karate_perm_truth,jazz_perm_truth,graph_perm_truth]


𝓗 = Modularity(1.0)

fn_louvain_mod = lambda G,_: louvain(G, 𝓗)
fn_leiden_mod  = lambda G,_: leiden(G, 𝓗)

algorithms = [ fn_louvain_mod, fn_leiden_mod]
algo_names = [ 'Louvain (Mod)', 'Leiden (Mod)']

lp_a_b =lambda agents, initial_clustering, pre: locally_popular_clustering_with_hop_distance(agents, f, e, initial_clustering,mode='B',pre = pre)
lp_a_f =lambda agents, initial_clustering, pre: locally_popular_clustering_with_hop_distance(agents, f, e, initial_clustering,mode='F',pre = pre)
lp_a_e =lambda agents, initial_clustering, pre: locally_popular_clustering_with_hop_distance(agents, f, e, initial_clustering,mode='E',pre = pre)

algorithms = [ lp_a_b,lp_a_f,lp_a_e,fn_louvain_mod, fn_leiden_mod]
algo_names = [ 'LP (Balanced) Heuristic',\
               'LP (Friend-Oriented) Heuristic','LP (Enemy-Averse) Heuristic',\
               'Louvain (Mod)', 'Leiden (Mod)']
is_lp_heuristic = [True,True,True, False, False]

collected_data = {}
for ((graph, g_name,clusters,truth), (algo, a_name,lp_heuristic)) in \
    itertools.product(zip(graphs, graph_names, expected_clusters,graph_truths), zip(algorithms, algo_names,is_lp_heuristic)):
    
        
        
    agents = []
    for i in range(len(graph)):
        agents += [list(graph[i].nodes())]


    if lp_heuristic:
        # start with everyone alone
        a_name_modified = a_name + ' starting with everyone alone'
        initial_clusters = len(agents[0])
        if graph == cora_perm_graph:
            a_name_modified += ' *6 starting clusters'
            initial_clusters = 6
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        
        test_callable = lambda a: algo(a,initial_clusters,None)
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

        
        # starting with predicted number of clusters
        a_name_modified = a_name + ' starting with predicted number of clusters'
        initial_clusters = clusters
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        
        test_callable = lambda a: algo(a,initial_clusters,None)
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

        
        # start with the output of leiden
        a_name_modified = a_name + ' starting with the output of leiden'
        initial_clusters = clusters
        print(f"Running {a_name_modified} on {g_name} … ", end='')
        
        test_callable = lambda a: algo(a,initial_clusters,fn_leiden_mod)
        times,outputs = time_tester(test_callable,graph)
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time

        collected_data[(a_name_modified,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))

       

    else:
        print(f"Running {a_name} on {g_name} … ", end='')
        test_callable = lambda a : algo(a,_)
        times,outputs = time_tester(test_callable,graph)
        outputs = [extract_labels_from_communities(c.communities) for c in outputs]
        avg_time = sum(times)/len(times)
        scores = calculate_scores_CD(outputs,truth,graph)
        scores['Time'] = avg_time
        collected_data[(a_name,g_name)] = scores
        print(f"execution time: ~ {avg_time:.6f} seconds.")
        for score_name in scores.keys():
            print(score_name,": ~",scores.get(score_name))


Running LP (Balanced) Heuristic starting with everyone alone *6 starting clusters on Cora … execution time: ~ 22.741460 seconds.
Rand Index : ~ 0.7126747275377202
Modularity : ~ 0.6119779480296788
Time : ~ 22.741460300050676
Running LP (Balanced) Heuristic starting with predicted number of clusters on Cora … execution time: ~ 24.870227 seconds.
Rand Index : ~ 0.7291995313861596
Modularity : ~ 0.6538964817962871
Time : ~ 24.870227199979126
Running LP (Balanced) Heuristic starting with the output of leiden on Cora … execution time: ~ 133.911857 seconds.
Rand Index : ~ 0.8003223766382795
Modularity : ~ 0.6267124964515555
Time : ~ 133.91185679996852
Running LP (Friend-Oriented) Heuristic starting with everyone alone *6 starting clusters on Cora … execution time: ~ 22.635672 seconds.
Rand Index : ~ 0.7126138863136712
Modularity : ~ 0.6173800898895958
Time : ~ 22.63567200000398
Running LP (Friend-Oriented) Heuristic starting with predicted number of clusters on Cora … execution time: ~ 24.07

## Gather the numbers

We can use the collected_data dictionairy to build a table for better comparison


In [4]:
print(collected_data)

{('LP (Balanced) Heuristic starting with everyone alone *6 starting clusters', 'Cora'): {'Rand Index': np.float64(0.7126747275377202), 'Modularity': 0.6119779480296788, 'Time': 22.741460300050676}, ('LP (Balanced) Heuristic starting with predicted number of clusters', 'Cora'): {'Rand Index': np.float64(0.7291995313861596), 'Modularity': 0.6538964817962871, 'Time': 24.870227199979126}, ('LP (Balanced) Heuristic starting with the output of leiden', 'Cora'): {'Rand Index': np.float64(0.8003223766382795), 'Modularity': 0.6267124964515555, 'Time': 133.91185679996852}, ('LP (Friend-Oriented) Heuristic starting with everyone alone *6 starting clusters', 'Cora'): {'Rand Index': np.float64(0.7126138863136712), 'Modularity': 0.6173800898895958, 'Time': 22.63567200000398}, ('LP (Friend-Oriented) Heuristic starting with predicted number of clusters', 'Cora'): {'Rand Index': np.float64(0.7288775912768417), 'Modularity': 0.6355939437599157, 'Time': 24.07152140000835}, ('LP (Friend-Oriented) Heuristi